In [1]:

import numpy as np
import torch
import glob
import cv2
import os
import os.path as osp
import pickle
import json
import pandas as pd

import numpy as np
import torch
import glob
import cv2
import os
import os.path as osp
import pickle
import csv
import numpy.linalg as LA
import data.transforms as T

/home/moon/.conda/envs/cuti2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def eul2rotm_ypr(euler):
    R_x = np.array(
        [
            [1, 0, 0],
            [0, np.cos(euler[0]), -np.sin(euler[0])],
            [0, np.sin(euler[0]), np.cos(euler[0])],
        ],
        dtype=np.float32,
    )

    R_y = np.array(
        [
            [np.cos(euler[1]), 0, np.sin(euler[1])],
            [0, 1, 0],
            [-np.sin(euler[1]), 0, np.cos(euler[1])],
        ],
        dtype=np.float32,
    )

    R_z = np.array(
        [
            [np.cos(euler[2]), -np.sin(euler[2]), 0],
            [np.sin(euler[2]), np.cos(euler[2]), 0],
            [0, 0, 1],
        ],
        dtype=np.float32,
    )

    return np.dot(R_z, np.dot(R_x, R_y))


def create_masks(image):
    masks = torch.zeros((1, height, width), dtype=torch.uint8)
    return masks


def read_line_file(filename, min_line_length=10):
    segs = []  # line segments
    # csv 파일 열어서 Line 정보 가져오기
    with open(filename, "r") as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            segs.append([float(row[0]), float(row[1]), float(row[2]), float(row[3])])
    segs = np.array(segs, dtype=np.float32)
    lengths = LA.norm(segs[:, 2:] - segs[:, :2], axis=1)
    segs = segs[lengths > min_line_length]
    return segs


def normalize_segs(segs, pp, rho):
    pp = np.array([pp[0], pp[1], pp[0], pp[1]], dtype=np.float32)
    return rho * (segs - pp)


def normalize_safe_np(v, axis=-1, eps=1e-6):
    de = LA.norm(v, axis=axis, keepdims=True)
    de = np.maximum(de, eps)
    return v / de


def segs2lines_np(segs):
    ones = np.ones(len(segs))
    ones = np.expand_dims(ones, axis=-1)
    p1 = np.concatenate([segs[:, :2], ones], axis=-1)
    p2 = np.concatenate([segs[:, 2:], ones], axis=-1)
    lines = np.cross(p1, p2)
    return normalize_safe_np(lines)


def sample_segs_np(segs, num_sample, use_prob=True):
    num_segs = len(segs)
    sampled_segs = np.zeros([num_sample, 4], dtype=np.float32)
    mask = np.zeros([num_sample, 1], dtype=np.float32)
    if num_sample > num_segs:
        sampled_segs[:num_segs] = segs
        mask[:num_segs] = np.ones([num_segs, 1], dtype=np.float32)
    else:
        lengths = LA.norm(segs[:, 2:] - segs[:, :2], axis=-1)
        prob = lengths / np.sum(lengths)
        idxs = np.random.choice(segs.shape[0], num_sample, replace=True, p=prob)
        sampled_segs = segs[idxs]
        mask = np.ones([num_sample, 1], dtype=np.float32)
    return sampled_segs, mask


def sample_vert_segs_np(segs, thresh_theta=22.5):
    lines = segs2lines_np(segs)
    (a, b) = lines[:, 0], lines[:, 1]
    theta = np.arctan2(np.abs(b), np.abs(a))
    thresh_theta = np.radians(thresh_theta)
    return segs[theta < thresh_theta]
def make_transform(self):
    return T.Compose(
        [T.ToTensor(), T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]
    )

In [3]:
path = "/home/moon/source/CuTi/matterport/mp3d_planercnn_json/cached_set_train.json"

In [4]:
root = "/home/moon/source/CuTi/matterport/"

In [5]:
with open(osp.join(path)) as f:
   split = json.load(f)

In [6]:
(split['data'][0]['0']['file_name']).split('/')[6:]

['rgb', '17DRP5sb8fy', '0_0_10.png']

In [7]:
scene_info = {
            "images": [],
            "poses": [],
            "intrinsics": [],
            "lines": [],
        }  # line 정보 추가
base_pose = np.array([0, 0, 0, 0, 0, 0, 1])

In [8]:
img_filename = []

In [9]:
target = {}
extra = {}

In [10]:
for i in range(2):
    images = []
    lines = []
    for imgnum in ["0", "1"]:
        img_name = os.path.join(
            root,
            "/".join(str(split["data"][i][imgnum]["file_name"]).split("/")[6:]),
        )
        line_name = img_name.split("/")
        line_name[8] = img_name.split("/")[8].split(".")[0] + "_line.csv"
        line_name = "/".join(line_name)

        images.append(img_name)
        lines.append(line_name)

    rel_pose = np.array(
        split["data"][i]["rel_pose"]["position"]
        + split["data"][i]["rel_pose"]["rotation"]
    )
    og_rel_pose = np.copy(rel_pose)

    # on matterport, we scale depths to balance rot & trans loss
    rel_pose[:3] /= 0.5
    cprp = np.copy(rel_pose)
    rel_pose[6] = cprp[
        3
    ]  # swap 3 & 6, we want W last for consistency with our other datasets
    rel_pose[3] = cprp[6]
    if rel_pose[6] < 0:  # normalize quaternions to have positive "W"
        rel_pose[3:] *= -1
    poses = np.vstack([base_pose, rel_pose])

    intrinsics = np.array(
        [[517.97, 517.97, 320, 240], [517.97, 517.97, 320, 240]]
    )  # 480 x 640 imgs

    scene_info["images"].append(images)
    scene_info["poses"] += [poses]
    scene_info["intrinsics"] += [intrinsics]
    scene_info["lines"].append(lines)

In [11]:
poses = scene_info["poses"][0]

In [12]:
poses

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 3.11850524e+00,  1.98698733e-01, -9.10757182e-01,
         5.83339888e-02,  1.73472348e-17,  3.00102356e-01,
         9.52121695e-01]])

In [13]:
poses = np.stack(poses).astype(np.float32)
poses = torch.from_numpy(poses)

In [14]:
import lietorch
from lietorch import SE3

In [15]:
def normalize_preds(self, Gs, pose_preds):
        pred_out_Gs = SE3(pose_preds)
        
        normalized = pred_out_Gs.data[:,:,3:].norm(dim=-1).unsqueeze(2)
        eps = torch.ones_like(normalized) * .01
        pred_out_Gs_new = SE3(torch.clone(pred_out_Gs.data))
        pred_out_Gs_new.data[:,:,3:] = pred_out_Gs.data[:,:,3:] / torch.max(normalized, eps)
        
        # print("shape____1",Gs[:,:1].data.shape)
        # print("shape____2",pred_out_Gs_new.data[:,1:].shape)
        
        these_out_Gs = SE3(torch.cat([Gs[:,:1].data, pred_out_Gs_new.data[:,1:]], dim=1))
            
        # if inference:
        #     out_Gs = these_out_Gs.data[0].cpu().numpy()
        # else:
        out_Gs = [these_out_Gs]

        return out_Gs

In [16]:
Ps = SE3(poses)
Gs = SE3.IdentityLike(Ps)
Ps_out = SE3(Ps.data.clone())

In [17]:
#ps_out == Ps
#Ps_est == Gs

In [18]:
ii, jj = torch.tensor([0, 1]), torch.tensor([1, 0])

In [19]:
Ps_out[:,[0,1]]

SE3: size=torch.Size([2]), device=cpu, dtype=torch.float32

In [20]:
Ps_out[:,ii]

SE3: size=torch.Size([2]), device=cpu, dtype=torch.float32

In [21]:
Ps_out2 = Ps_out[:,ii].inv()

SE3: size=torch.Size([2]), device=cpu, dtype=torch.float32

In [22]:
dP = Ps_out[:,jj] * Ps_out2

: 

: 

In [ ]:
dP

In [ ]:
dP = torch.from_numpy(dP)

In [ ]:
dP

In [ ]:
dP.inv()